In [2]:
## 直接提取原来omega数据

import xml.etree.ElementTree as ET

def parse_xrdml_phi_0_to_355(input_file, output_file):
    tree = ET.parse(input_file)
    root = tree.getroot()
    ns = {'ns': root.tag.split('}')[0].strip('{')}

    output_lines = []
    phi_values = list(range(0, 360, 5))  # 共 72 个点

    for scan in root.findall('.//ns:scan', ns):
        omega_elem = scan.find('.//ns:positions[@axis="Omega"]/ns:commonPosition', ns)
        if omega_elem is None:
            continue
        omega = float(omega_elem.text)


        intensity_elem = scan.find('.//ns:intensities', ns)
        if intensity_elem is None:
            continue
        intensities = list(map(float, intensity_elem.text.strip().split()))

        if len(intensities) != 73:
            print(f"⚠️ Omega={omega}: intensity 点数 {len(intensities)} ≠ 73，跳过")
            continue

        intensities = intensities[:-1]  # 保留前 72 个

        phi_reordered = [(i + 180) % 360 for i in range(-180, 180, 5)] 
        reordered_data = sorted(zip(phi_reordered, intensities), key=lambda x: x[0])

        for phi, inten in reordered_data:
            output_lines.append(f"{omega:.3f}\t{phi:.1f}\t{inten:.2f}")

    with open(output_file, 'w') as f:
        f.write("Omega(deg)\tPhi(deg)\tIntensity(counts)\n")
        f.write('\n'.join(output_lines))

    print(f"✅ 已生成修正后的极图数据文件：{output_file}")

# 使用示例
if __name__ == "__main__":
    input_xrdml = "/Users/maguag/Desktop/XRD教学/XRDML/PKU20250731/220.xrdml"  # 替换为你的文件路径
    output_txt = "/Users/maguag/Desktop/XRD教学/XRDML/PKU20250731/220-220.txt"   # 替换为你的文件路径
    parse_xrdml_phi_0_to_355(input_xrdml, output_txt)

✅ 已生成修正后的极图数据文件：/Users/maguag/Desktop/XRD教学/XRDML/PKU20250731/220-220.txt


In [2]:
#提取处理后的omega数据，添加已经知的正确角度


import xml.etree.ElementTree as ET

def parse_xrdml_to_polefig(input_file, output_file):
    tree = ET.parse(input_file)
    root = tree.getroot()
    ns = {'ns': root.tag.split('}')[0].strip('{')}
    
    output_lines = []
    phi_values = list(range(0, 360, 5))     
    fixed_omega = list(range(-5, 61, 5))   # 如果这里因为已经知道角度，直接用近似的数据代替了
    
    scans = root.findall('.//ns:scan', ns)
    if len(scans) < len(fixed_omega):
        print(f"警告：扫描块数({len(scans)})少于预期的 Omega 数({len(fixed_omega)})！")
    
    for idx, scan in enumerate(scans):
        if idx >= len(fixed_omega):
            break
        omega = fixed_omega[idx]     
        intensity_elem = scan.find('.//ns:intensities', ns)
        if intensity_elem is None:
            continue
        intensities = list(map(float, intensity_elem.text.strip().split())) 
        
        if len(intensities) != 73:
            print(f"警告：在 Omega={omega}° 时 intensity 数量({len(intensities)}) != 73，跳过此块。")
            continue
        intensities = intensities[:-1]  
        
        if len(intensities) != len(phi_values):
            print(f"警告：在 Omega={omega}° 时 intensity 数量({len(intensities)}) 与 phi 数量({len(phi_values)})不匹配。")
            continue
        
        for phi, inten in zip(phi_values, intensities):
            output_lines.append(f"{omega}\t{phi}\t{inten}")
    
    with open(output_file, "w") as f:
        f.write("Omega(deg)\tPhi(deg)\tIntensity(counts)\n")
        f.write("\n".join(output_lines))
    
    print(f"✅ 已成功生成文件：{output_file}")



# 示例调用
if __name__ == "__main__":
    input_xrdml = "/Users/maguag/Desktop/20250731/220.xrdml"  # 替换为你的文件路径
    output_txt = "/Users/maguag/Desktop/20250731/220.txt"     # 替换为你的文件路径
    parse_xrdml_to_polefig(input_xrdml, output_txt)

✅ 已成功生成文件：/Users/maguag/Desktop/20250731/220.txt
